In [1]:
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader

/mnt/disk1/sumdev/CourseWork/a3_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load training dataset, only the first 50,000 examples
train_dataset = load_dataset("wmt16", "de-en", split="train[:50000]")

# Load validation and test datasets
validation_dataset = load_dataset("wmt16", "de-en", split="validation")
test_dataset = load_dataset("wmt16", "de-en", split="test")
print(len(train_dataset), len(validation_dataset), len(test_dataset))

50000 2169 2999


# Setup-2A

In [4]:
import torch
import math
import torch.nn as nn
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import Multi30k
from typing import Iterable, List

# Define the PositionalEncoding module
class PositionalEncoding(nn.Module):
    def __init__(self, emb_size: int, dropout: float, maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])


class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)


# Define the Transformer model
class TransformerModel(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, emb_dim=234, nhead=6, num_encoder_layers=5, num_decoder_layers=5, dropout=0.1):
        super(TransformerModel, self).__init__()
        self.embedding_dim = emb_dim
        self.transformer = nn.Transformer(
            d_model=emb_dim, nhead=nhead, num_encoder_layers=num_encoder_layers, 
            num_decoder_layers=num_decoder_layers, dropout=dropout, batch_first=True)
        self.src_token_emb = nn.Embedding(src_vocab_size, emb_dim)
        self.tgt_token_emb = nn.Embedding(tgt_vocab_size, emb_dim)
        self.linear = nn.Linear(emb_dim, tgt_vocab_size)
        self.positional_encoding = PositionalEncoding(emb_dim, dropout=dropout)


    def forward(self, src, tgt, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, memory_padding_mask):
        src_emb = self.positional_encoding(self.src_token_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_token_emb(tgt))
        output = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None, src_padding_mask, tgt_padding_mask, memory_padding_mask)
        output = self.linear(output)
        return output
    
    def encode(self, src, src_mask):
        return self.transformer.encoder(self.positional_encoding(self.src_token_emb(src)), src_mask)

    def decode(self, tgt, memory, tgt_mask):
        return self.transformer.decoder(self.positional_encoding(self.tgt_token_emb(tgt)), memory, tgt_mask)


In [5]:
from typing import Iterable

# Define function to yield tokens from the dataset
def yield_tokens(data_iter: Iterable, language: str):
    for data_sample in data_iter:
        yield token_transform[language](data_sample['translation'][language])

# Set up tokenizers
token_transform = {}
SRC_LANGUAGE = 'de'
TGT_LANGUAGE = 'en'
token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='de_core_news_sm')
token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')


# Build vocabulary
vocab_transform = {}
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
vocab_transform[SRC_LANGUAGE] = build_vocab_from_iterator(yield_tokens(train_dataset, 'de'), specials=["<unk>", "<pad>", "<bos>", "<eos>"])
vocab_transform[TGT_LANGUAGE] = build_vocab_from_iterator(yield_tokens(train_dataset, 'en'), specials=["<unk>", "<pad>", "<bos>", "<eos>"])
vocab_transform[SRC_LANGUAGE].set_default_index(UNK_IDX)
vocab_transform[TGT_LANGUAGE].set_default_index(UNK_IDX)

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=device)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def create_mask(src, tgt):
    src_seq_len = src.shape[1]
    tgt_seq_len = tgt.shape[1]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=device).type(torch.bool)
    src_padding_mask = (src == PAD_IDX)
    tgt_padding_mask = (tgt == PAD_IDX)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize model
SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])
EMB_DIM = 256
N_HEAD = 8
NUM_ENCODER_LAYERS = 8
NUM_DECODER_LAYERS = 8
DROPOUT = 0.05
model_loaded = TransformerModel(src_vocab_size=SRC_VOCAB_SIZE, tgt_vocab_size=TGT_VOCAB_SIZE, 
            emb_dim=EMB_DIM, nhead=N_HEAD, num_encoder_layers=NUM_ENCODER_LAYERS, 
            num_decoder_layers=NUM_DECODER_LAYERS, dropout=DROPOUT)
model_loaded = model_loaded.to(device)
model_loaded.load_state_dict(torch.load('q2a_transformer_model.pth'))

<All keys matched successfully>

In [9]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(device)
    src_mask = src_mask.to(device)
    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(device)
    for i in range(max_len-1):
        memory = memory.to(device)
        tgt_mask = (generate_square_subsequent_mask(ys.size(1))
                    .type(torch.bool)).to(device)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.linear(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word[-1].item()
        ys = torch.cat([ys, torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
        if next_word == EOS_IDX:
            break
    return ys


# actual function to translate input sentence into target language
def translate(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = token_transform[SRC_LANGUAGE](src_sentence)
    src = torch.tensor([[vocab_transform[SRC_LANGUAGE][token] for token in src]], dtype=torch.long)
    num_tokens = src.shape[1]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")


In [10]:
import sacrebleu
from nltk.translate import meteor_score
from torchtext.data.utils import get_tokenizer
from nltk.tokenize import word_tokenize
from bert_score import score
import nltk
nltk.download('wordnet')
nltk.download('punkt')


tokenizer = get_tokenizer("basic_english")

def calculate_bleu(reference_corpus, predicted_corpus):
    return sacrebleu.corpus_bleu(predicted_corpus, [reference_corpus])

def calculate_meteor(reference_corpus, predicted_corpus):
    m_score = 0
    for line in zip(reference_corpus, predicted_corpus):
        ref = word_tokenize(line[0])
        hypo = word_tokenize(line[1])
        # m_score += meteor_score.meteor_score([ref], hypo)
        m_score += meteor_score.single_meteor_score(ref, hypo)
    return m_score / len(reference_corpus)

def calculate_bert_score(reference_corpus, predicted_corpus):
    _, _, bert_scores = score(predicted_corpus, reference_corpus, lang="en", rescale_with_baseline=True)
    return bert_scores.mean().item()

[nltk_data] Downloading package wordnet to
[nltk_data]     /mnt/disk1/sumdev/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /mnt/disk1/sumdev/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [16]:
sentence_german_val, sentence_english_val, sentence_english_translated_val = [], [], []

for i in range(len(validation_dataset)):
    sentence_german_val.append(validation_dataset[i]['translation']['de'])
    sentence_english_val.append(validation_dataset[i]['translation']['en'])
    sentence_english_translated_val.append(translate(model_loaded, validation_dataset[i]['translation']['de']))

bleu_score_val = calculate_bleu(sentence_english_translated_val, sentence_english_val)
meteor_score_val = calculate_meteor(sentence_english_translated_val, sentence_english_val)
bert_score_val = calculate_bert_score(sentence_english_translated_val, sentence_english_val)

print(f'BLEU Score: {bleu_score_val.score}')
print(f'BLEU-1 Score: {bleu_score_val.precisions[0]}')
print(f'BLEU-2 Score: {bleu_score_val.precisions[1]}')
print(f'BLEU-3 Score: {bleu_score_val.precisions[2]}')
print(f'BLEU-4 Score: {bleu_score_val.precisions[3]}')
print(f'METEOR Score: {meteor_score_val}')
print(f'BERT Score: {bert_score_val}')

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BLEU Score: 0.9478835493638972
BLEU-1 Score: 23.710785263656526
BLEU-2 Score: 3.0627456294890907
BLEU-3 Score: 0.3182368727289999
BLEU-4 Score: 0.060082613593691325
METEOR Score: 0.12734655510735712
BERT Score: -0.04137616604566574


In [17]:
sentence_german_test, sentence_english_test, sentence_english_translated_test = [], [], []

for i in range(len(test_dataset)):
    sentence_german_test.append(test_dataset[i]['translation']['de'])
    sentence_english_test.append(test_dataset[i]['translation']['en'])
    sentence_english_translated_test.append(translate(model_loaded, test_dataset[i]['translation']['de']))

bleu_score_test = calculate_bleu(sentence_english_translated_test, sentence_english_test)
meteor_score_test = calculate_meteor(sentence_english_translated_test, sentence_english_test)
bert_score_test = calculate_bert_score(sentence_english_translated_test, sentence_english_test)

print(f'BLEU Score: {bleu_score_test.score}')
print(f'BLEU-1 Score: {bleu_score_test.precisions[0]}')
print(f'BLEU-2 Score: {bleu_score_test.precisions[1]}')
print(f'BLEU-3 Score: {bleu_score_test.precisions[2]}')
print(f'BLEU-4 Score: {bleu_score_test.precisions[3]}')
print(f'METEOR Score: {meteor_score_test}')
print(f'BERT Score: {bert_score_test}')

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BLEU Score: 1.004693931335485
BLEU-1 Score: 23.599868993590043
BLEU-2 Score: 2.9466623036649215
BLEU-3 Score: 0.3475507991947833
BLEU-4 Score: 0.0798214901220906
METEOR Score: 0.12672719694687268
BERT Score: -0.05438026785850525


# Setup-2B

In [18]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = T5Tokenizer.from_pretrained("google-t5/t5-small")
model = T5ForConditionalGeneration.from_pretrained("google-t5/t5-small")
model = model.to(device)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [19]:
import sacrebleu
from nltk.translate import meteor_score
from nltk.tokenize import word_tokenize
from bert_score import score
import nltk
nltk.download('wordnet')
nltk.download('punkt')



def calculate_bleu(reference_corpus, predicted_corpus):
    return sacrebleu.corpus_bleu(predicted_corpus, [reference_corpus])

def calculate_meteor(reference_corpus, predicted_corpus):
    m_score = 0
    for line in zip(reference_corpus, predicted_corpus):
        ref = word_tokenize(line[0])
        hypo = word_tokenize(line[1])
        # m_score += meteor_score.meteor_score([ref], hypo)
        m_score += meteor_score.single_meteor_score(ref, hypo)
    return m_score / len(reference_corpus)

def calculate_bert_score(reference_corpus, predicted_corpus):
    _, _, bert_scores = score(predicted_corpus, reference_corpus, lang="de", rescale_with_baseline=True)
    return bert_scores.mean().item()

[nltk_data] Downloading package wordnet to
[nltk_data]     /mnt/disk1/sumdev/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /mnt/disk1/sumdev/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [23]:
## Translate English to German
def translate_engligh_to_german(dataset):
    reference_corpus = []
    predicted_corpus = []
    prefix = "translate English to German: "
    
    for example in dataset:
        input_text = prefix + example["translation"]["en"]
        input_ids = tokenizer(input_text, return_tensors="pt").input_ids
        input_ids = input_ids.to(device)
        outputs = model.generate(input_ids)
        predicted_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        reference_corpus.append(example["translation"]["de"])
        predicted_corpus.append(predicted_text)
    return reference_corpus, predicted_corpus

In [24]:
reference_corpus_val, predicted_corpus_val = translate_engligh_to_german(validation_dataset)

bleu_score_ = calculate_bleu(reference_corpus_val, predicted_corpus_val)
meteor_score_ = calculate_meteor(reference_corpus_val, predicted_corpus_val)
bert_score_ = calculate_bert_score(reference_corpus_val, predicted_corpus_val)

print(f'BLEU Score: {bleu_score_.score}')
print(f'BLEU-1: {bleu_score_.precisions[0]}')
print(f'BLEU-2: {bleu_score_.precisions[1]}')
print(f'BLEU-3: {bleu_score_.precisions[2]}')
print(f'BLEU-4: {bleu_score_.precisions[3]}')
print(f'METEOR Score: {meteor_score_}')
print(f'BERT Score: {bert_score_}')

/mnt/disk1/sumdev/CourseWork/a3_env/lib/python3.12/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


BLEU Score: 12.756317218851342
BLEU-1: 59.53264548754129
BLEU-2: 33.98801002147459
BLEU-3: 21.935995244228675
BLEU-4: 14.930093209054593
METEOR Score: 0.3697782590377402
BERT Score: 0.46585261821746826


In [25]:
reference_corpus_test, predicted_corpus_test = translate_engligh_to_german(test_dataset)

bleu_score_ = calculate_bleu(reference_corpus_test, predicted_corpus_test)
meteor_score_ = calculate_meteor(reference_corpus_test, predicted_corpus_test)
bert_score_ = calculate_bert_score(reference_corpus_test, predicted_corpus_test)

print(f'BLEU Score: {bleu_score_.score}')
print(f'BLEU-1: {bleu_score_.precisions[0]}')
print(f'BLEU-2: {bleu_score_.precisions[1]}')
print(f'BLEU-3: {bleu_score_.precisions[2]}')
print(f'BLEU-4: {bleu_score_.precisions[3]}')
print(f'METEOR Score: {meteor_score_}')
print(f'BERT Score: {bert_score_}')

/mnt/disk1/sumdev/CourseWork/a3_env/lib/python3.12/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


BLEU Score: 13.841273518987153
BLEU-1: 61.59426598183454
BLEU-2: 37.238573517258914
BLEU-3: 24.709271686048556
BLEU-4: 17.110997683458322
METEOR Score: 0.3889479564070389
BERT Score: 0.480898916721344


# Setup-2C

In [30]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, T5Config, AutoTokenizer
from torch.utils.data import Dataset, DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the T5 tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small")
model_loaded = T5ForConditionalGeneration.from_pretrained("google-t5/t5-small")
model_loaded = model_loaded.to(device)
model_loaded.load_state_dict(torch.load('q2c_t5_model.pth'))

<All keys matched successfully>

In [31]:
import sacrebleu
from nltk.translate import meteor_score
from torchtext.data.utils import get_tokenizer
from nltk.tokenize import word_tokenize
from bert_score import score
import nltk
nltk.download('wordnet')
nltk.download('punkt')

def calculate_bleu(reference_corpus, predicted_corpus):
    return sacrebleu.corpus_bleu(predicted_corpus, [reference_corpus])

def calculate_meteor(reference_corpus, predicted_corpus):
    m_score = 0
    for line in zip(reference_corpus, predicted_corpus):
        ref = word_tokenize(line[0])
        hypo = word_tokenize(line[1])
        # m_score += meteor_score.meteor_score([ref], hypo)
        m_score += meteor_score.single_meteor_score(ref, hypo)
    return m_score / len(reference_corpus)

def calculate_bert_score(reference_corpus, predicted_corpus):
    _, _, bert_scores = score(predicted_corpus, reference_corpus, lang="en", rescale_with_baseline=True)
    return bert_scores.mean().item()

def translate_sentence(finetuned_model, dataset):
    finetuned_model = finetuned_model.to(device)
    sentence_german = []
    sentence_english = []
    sentence_english_translated = []
    for i in range(len(dataset)):
        sentence_german.append(dataset[i]['translation']['de'])
        sentence_english.append(dataset[i]['translation']['en'])
        input_ids = tokenizer(sentence_german[i], return_tensors="pt").input_ids
        input_ids = input_ids.to(device)
        outputs = finetuned_model.generate(input_ids)
        sentence_english_translated.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
    return sentence_german, sentence_english, sentence_english_translated

[nltk_data] Downloading package wordnet to
[nltk_data]     /mnt/disk1/sumdev/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /mnt/disk1/sumdev/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [32]:
sentence_german_val, sentence_english_val, sentence_english_translated_val = translate_sentence(model_loaded, validation_dataset)

bleu_score_val = calculate_bleu(sentence_english_translated_val, sentence_english_val)
meteor_score_val = calculate_meteor(sentence_english_translated_val, sentence_english_val)
bert_score_val = calculate_bert_score(sentence_english_translated_val, sentence_english_val)

print(f'BLEU Score: {bleu_score_val.score}')
print(f'BLEU-1 Score: {bleu_score_val.precisions[0]}')
print(f'BLEU-2 Score: {bleu_score_val.precisions[1]}')
print(f'BLEU-3 Score: {bleu_score_val.precisions[2]}')
print(f'BLEU-4 Score: {bleu_score_val.precisions[3]}')
print(f'METEOR Score: {meteor_score_val}')
print(f'BERT Score: {bert_score_val}')

/mnt/disk1/sumdev/CourseWork/a3_env/lib/python3.12/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BLEU Score: 9.796981308712855
BLEU-1 Score: 31.08972288611847
BLEU-2 Score: 13.104756742105977
BLEU-3 Score: 6.507231576697461
BLEU-4 Score: 3.474777819501815
METEOR Score: 0.4084099646621174
BERT Score: 0.33497461676597595


In [33]:
sentence_german_test, sentence_english_test, sentence_english_translated_test = translate_sentence(model_loaded, test_dataset)

bleu_score_test = calculate_bleu(sentence_english_translated_test, sentence_english_test)
meteor_score_test = calculate_meteor(sentence_english_translated_test, sentence_english_test)
bert_score_test = calculate_bert_score(sentence_english_translated_test, sentence_english_test)

print(f'BLEU Score: {bleu_score_test.score}')
print(f'BLEU-1 Score: {bleu_score_test.precisions[0]}')
print(f'BLEU-2 Score: {bleu_score_test.precisions[1]}')
print(f'BLEU-3 Score: {bleu_score_test.precisions[2]}')
print(f'BLEU-4 Score: {bleu_score_test.precisions[3]}')
print(f'METEOR Score: {meteor_score_test}')
print(f'BERT Score: {bert_score_test}')

/mnt/disk1/sumdev/CourseWork/a3_env/lib/python3.12/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BLEU Score: 11.13598085929917
BLEU-1 Score: 32.971506105834465
BLEU-2 Score: 14.833115183246074
BLEU-3 Score: 7.63235319419831
BLEU-4 Score: 4.119877365165176
METEOR Score: 0.43761230509235594
BERT Score: 0.34816470742225647
